### Dart Open Api 사용해서 크롤링

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import json
import requests

In [3]:
# 개발 참조 https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003

### 상장된 회사들만 조회하기 위해, 상장회사의 코드를 통해 공시고유번호를 추출한다.

In [4]:
# 상장된 회사들의 회사코드

In [5]:
df_listed = pd.read_csv("상장회사목록.csv", encoding = "ms949")

상장기업 = df_listed["상장코드"]
def delete_A(list_code):
    new_code = list_code[1:]
    return new_code

df_listed["상장코드"] = 상장기업.apply(delete_A)
상장코드 = df_listed["상장코드"]

In [6]:
# 회사공시고유번호정보

In [7]:
df_crpcode = pd.read_csv("회사별 공시고유번호.csv", encoding = "ms949")
df_crpcode = df_crpcode.drop("Unnamed: 0", axis = 1)

In [8]:
# 공시고유번호는 무조건 8자리여야 하기 때문에 8자리 아닌 것들은 앞에 0을 붙여줌
def make_eight_digits(crpcode):
    crpcode = str(crpcode)
    code_length = len(crpcode)
    add_zeros = 8 - code_length
    prefix = add_zeros * str(0)
    return(prefix + crpcode)
    

df_crpcode["공시고유번호"] = df_crpcode["공시고유번호"].apply(make_eight_digits)

#### 상장코드가 존재하는 회사들의 공시고유번호만 뽑아와서 데이터프레임을 만듦

In [9]:
old = pd.DataFrame()
for company in 상장코드:
    new = df_crpcode[df_crpcode["상장코드"] == company]
    old = pd.concat([old, new])

In [10]:
old.head()

,공시고유번호,상장코드,최종수정일,회사명
67511,00126380,005930,20191209,삼성전자
71468,00164779,000660,20191206,SK하이닉스
70948,00877059,207940,20200113,삼성바이오로직스
72717,00266961,035420,20200114,NAVER
75127,00164742,005380,20200110,현대자동차


In [61]:
# 요청값
# 내 API key니까 다들 하나씩 발급받으면 좋을듯!
# 1일 1만회 요청건수 제한
auth_key="847436c641de81093f023d156e8686df4e226471" 
# 사업연도 설정
사업연도 ="2018"
# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011
report_code = "11014"

In [98]:
crp_code ="00356370" 

In [99]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key={0}&corp_code={1}&bsns_year={2}&reprt_code={3}"
url = url.format(auth_key, crp_code, 사업연도, report_code)

In [100]:
response = requests.get(url)
data = json.loads(response.content)

In [101]:
data

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '11014',
   'corp_code': '00356370',
   'stock_code': '051900',
   'account_nm': '유동자산',
   'fs_div': 'CFS',
   'fs_nm': '연결재무제표',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'thstrm_nm': '제 18 기3분기말',
   'thstrm_dt': '2018.09.30 현재',
   'thstrm_amount': '1,684,769,908,952',
   'frmtrm_nm': '제 17 기말',
   'frmtrm_dt': '2017.12.31 현재',
   'frmtrm_amount': '1,513,604,950,809',
   'ord': '1'},
  {'rcept_no': '11014',
   'corp_code': '00356370',
   'stock_code': '051900',
   'account_nm': '비유동자산',
   'fs_div': 'CFS',
   'fs_nm': '연결재무제표',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'thstrm_nm': '제 18 기3분기말',
   'thstrm_dt': '2018.09.30 현재',
   'thstrm_amount': '3,513,141,481,409',
   'frmtrm_nm': '제 17 기말',
   'frmtrm_dt': '2017.12.31 현재',
   'frmtrm_amount': '3,264,915,009,903',
   'ord': '3'},
  {'rcept_no': '11014',
   'corp_code': '00356370',
   'stock_code': '051900',
   'account_nm': '자산총계',
   'fs_div': 'CFS',
   'fs_nm': '연결재무

In [102]:
# 문제가 2019년 3분기가 다운로드가 안 됨--- 왜 안되는지 모르겠네..

# -------------------------------------------------------------- ## 

In [103]:
data["list"]

[{'rcept_no': '11014',
  'corp_code': '00356370',
  'stock_code': '051900',
  'account_nm': '유동자산',
  'fs_div': 'CFS',
  'fs_nm': '연결재무제표',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'thstrm_nm': '제 18 기3분기말',
  'thstrm_dt': '2018.09.30 현재',
  'thstrm_amount': '1,684,769,908,952',
  'frmtrm_nm': '제 17 기말',
  'frmtrm_dt': '2017.12.31 현재',
  'frmtrm_amount': '1,513,604,950,809',
  'ord': '1'},
 {'rcept_no': '11014',
  'corp_code': '00356370',
  'stock_code': '051900',
  'account_nm': '비유동자산',
  'fs_div': 'CFS',
  'fs_nm': '연결재무제표',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'thstrm_nm': '제 18 기3분기말',
  'thstrm_dt': '2018.09.30 현재',
  'thstrm_amount': '3,513,141,481,409',
  'frmtrm_nm': '제 17 기말',
  'frmtrm_dt': '2017.12.31 현재',
  'frmtrm_amount': '3,264,915,009,903',
  'ord': '3'},
 {'rcept_no': '11014',
  'corp_code': '00356370',
  'stock_code': '051900',
  'account_nm': '자산총계',
  'fs_div': 'CFS',
  'fs_nm': '연결재무제표',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'thstrm_nm': '제 18 기3분기말',
  'thstr

In [104]:
for i in data["list"] :
    if i["fs_div"] == "CFS": # 연결기준
        print("A" + i["stock_code"],i["account_nm"], i["thstrm_amount"])

A051900 유동자산 1,684,769,908,952
A051900 비유동자산 3,513,141,481,409
A051900 자산총계 5,197,911,390,361
A051900 유동부채 1,257,604,166,737
A051900 비유동부채 441,660,200,507
A051900 부채총계 1,699,264,367,244
A051900 자본금 88,589,470,000
A051900 이익잉여금 3,472,558,353,292
A051900 자본총계 3,498,647,023,117
A051900 매출액 1,737,237,665,409
A051900 영업이익 277,539,279,012
A051900 법인세차감전 순이익 271,193,825,086
A051900 당기순이익 207,172,999,001
A051900 당기순이익 207,172,999,001
